In [14]:
# Standard library imports
import os
import warnings
import gc
import json
from datetime import datetime
from pathlib import Path
import re
from collections import Counter
from typing import List, Dict, Tuple
import pickle
import joblib
from joblib import Parallel, delayed
import hashlib
import pyarrow.parquet as pq
import pyarrow as pa

# Core data science libraries
import numpy as np
import pandas as pd
from math import ceil
from scipy import stats

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Progress bar
from tqdm import tqdm

# Scikit-learn imports
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, classification_report, confusion_matrix,
    mean_absolute_percentage_error, precision_recall_curve
)
from sklearn.preprocessing import LabelEncoder, StandardScaler, PowerTransformer, QuantileTransformer, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.utils.class_weight import compute_class_weight
from sklearn.inspection import permutation_importance
from sklearn.impute import KNNImputer

# Catboost
import catboost as cb
from catboost import CatBoostRegressor, CatBoostClassifier

# Optuna for hyperparameter optimization
import optuna
from optuna.samplers import TPESampler

# Added tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras import backend as K

import multiprocessing

# Configuration
warnings.filterwarnings('ignore')

In [15]:
# Define the plot output path
plot_output_path = Path('plots')

# Create directory if it doesn't exist
plot_output_path.mkdir(exist_ok=True)
print(f"Directory '{plot_output_path}' is ready.")

# Define processed data output path
processed_data_output_path = Path('processed_data')

# Create directory if it doesn't exist
processed_data_output_path.mkdir(exist_ok=True)
print(f"Directory '{processed_data_output_path}' is ready.")

model_output_path = Path('best_model')
model_output_path.mkdir(exist_ok=True)

Directory 'plots' is ready.
Directory 'processed_data' is ready.


In [16]:
# Optimized physics calculations using vectorization
def remove_gravity_from_acc_vectorized(acc_values, quat_values):
    """Vectorized gravity removal for better performance"""
    num_samples = acc_values.shape[0]
    linear_accel = acc_values.copy()
    
    # Filter valid quaternions
    valid_mask = ~(np.any(np.isnan(quat_values), axis=1) | 
                   np.all(np.isclose(quat_values, 0), axis=1))
    
    if np.any(valid_mask):
        # Process all valid quaternions at once
        valid_quats = quat_values[valid_mask]
        
        # Batch rotation computation
        try:
            rotations = R.from_quat(valid_quats)
            gravity_world = np.array([0, 0, 9.81])
            
            # Apply rotations in batch
            gravity_sensor_frames = rotations.apply(gravity_world, inverse=True)
            linear_accel[valid_mask] = acc_values[valid_mask] - gravity_sensor_frames
        except:
            pass
            
    return linear_accel

In [17]:
def calculate_angular_velocity_vectorized(quat_values, time_delta=1/200):
    """Vectorized angular velocity calculation"""
    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))
    
    if num_samples < 2:
        return angular_vel
        
    # Process in chunks for memory efficiency
    chunk_size = 1000
    for start in range(0, num_samples - 1, chunk_size):
        end = min(start + chunk_size, num_samples - 1)
        
        q_t = quat_values[start:end]
        q_t_plus_dt = quat_values[start+1:end+1]
        
        # Find valid pairs
        valid_mask = ~(np.any(np.isnan(q_t), axis=1) | 
                      np.all(np.isclose(q_t, 0), axis=1) |
                      np.any(np.isnan(q_t_plus_dt), axis=1) | 
                      np.all(np.isclose(q_t_plus_dt, 0), axis=1))
        
        if np.any(valid_mask):
            try:
                valid_indices = np.where(valid_mask)[0]
                rot_t = R.from_quat(q_t[valid_mask])
                rot_t_plus_dt = R.from_quat(q_t_plus_dt[valid_mask])
                
                # Batch computation
                delta_rot = rot_t.inv() * rot_t_plus_dt
                angular_vel[start + valid_indices] = delta_rot.as_rotvec() / time_delta
            except:
                pass
                
    return angular_vel


In [20]:
# Parallel processing for sequence features
def process_sequence(seq_data):
    """Process a single sequence with all feature engineering"""
    seq_data = seq_data.copy()
    
    # Get numpy arrays for faster processing
    acc_values = seq_data[['acc_x', 'acc_y', 'acc_z']].values
    quat_values = seq_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values

    try:
        rotations = R.from_quat(quat_values)
        euler_angles = rotations.as_euler('xyz', degrees=True)
        seq_data['euler_x'] = euler_angles[:, 0]
        seq_data['euler_y'] = euler_angles[:, 1]
        seq_data['euler_z'] = euler_angles[:, 2]
    except:
        seq_data['euler_x'] = 0
        seq_data['euler_y'] = 0
        seq_data['euler_z'] = 0
    
    # Linear acceleration
    linear_accel = remove_gravity_from_acc_vectorized(acc_values, quat_values)
    seq_data['linear_acc_x'] = linear_accel[:, 0]
    seq_data['linear_acc_y'] = linear_accel[:, 1]
    seq_data['linear_acc_z'] = linear_accel[:, 2]

    # Acceleration features focusing on z and y axes
    seq_data['acc_yz_mag'] = np.sqrt(seq_data['acc_y']**2 + seq_data['acc_z']**2)
    seq_data['acc_y_z_ratio'] = seq_data['acc_y'] / (seq_data['acc_z'] + 1e-8)
    
    # Magnitudes
    seq_data['acc_mag'] = np.linalg.norm(acc_values, axis=1)
    seq_data['linear_acc_mag'] = np.linalg.norm(linear_accel, axis=1)
    
    # Jerk (simplified)
    seq_data['linear_acc_mag_jerk'] = np.gradient(seq_data['linear_acc_mag']) * 200
    
    # Angular velocity
    angular_vel = calculate_angular_velocity_vectorized(quat_values)
    seq_data['angular_vel_x'] = angular_vel[:, 0]
    seq_data['angular_vel_y'] = angular_vel[:, 1]
    seq_data['angular_vel_z'] = angular_vel[:, 2]
    
    # ToF aggregations (vectorized)
    for i in range(1, 6):
        pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]
        tof_data = seq_data[pixel_cols].values
        tof_data[tof_data == -1] = np.nan
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            seq_data[f'tof_{i}_mean'] = np.nanmean(tof_data, axis=1)
            seq_data[f'tof_{i}_std'] = np.nanstd(tof_data, axis=1)
            seq_data[f'tof_{i}_min'] = np.nanmin(tof_data, axis=1)
            seq_data[f'tof_{i}_max'] = np.nanmax(tof_data, axis=1)
            seq_data[f'tof_{i}_var'] = np.nanvar(tof_data, axis=1)

    # Add rotation-specific features for rot_z and rot_w
    seq_data['rot_z_w_product'] = seq_data['rot_z'] * seq_data['rot_w']
    seq_data['rot_z_w_ratio'] = seq_data['rot_z'] / (seq_data['rot_w'] + 1e-8)

    # Rolling statistics for important features (window of 10 samples = 50ms)
    for col in ['rot_z', 'rot_w', 'acc_z', 'acc_y', 'thm_2']:
        seq_data[f'{col}_rolling_mean'] = seq_data[col].rolling(10, center=True).mean()
        seq_data[f'{col}_rolling_std'] = seq_data[col].rolling(10, center=True).std()
        seq_data[f'{col}_diff'] = seq_data[col].diff()
    
    # Thermopile 2 specific features
    seq_data['thm_2_normalized'] = (seq_data['thm_2'] - seq_data['thm_2'].mean()) / (seq_data['thm_2'].std() + 1e-8)
    seq_data['thm_2_delta_from_mean'] = seq_data['thm_2'] - seq_data[['thm_1', 'thm_2', 'thm_3', 'thm_4', 'thm_5']].mean(axis=1)
    
    # ToF band statistics for tof_1 and tof_2 (statistically significant bands)
    bands_tof_1_2 = [
        (3, 7),    # Band 1: v[3-7]
        (11, 15),  # Band 2: v[11-15]
        (19, 23),  # Band 3: v[19-23]
        (27, 31),  # Band 4: v[27-31]
        (35, 39),  # Band 5: v[35-39]
        (43, 47),  # Band 6: v[43-47]
        (51, 55),  # Band 7: v[51-55]
        (59, 63),  # Band 8: v[59-63]
    ]
    
    for tof_num in [1, 2]:  # Only process tof_1 and tof_2
        for band_idx, (start, end) in enumerate(bands_tof_1_2, 1):
            # Get columns for this band (inclusive range)
            band_cols = [f"tof_{tof_num}_v{p}" for p in range(start, end + 1)]
            band_data = seq_data[band_cols].values
            band_data[band_data == -1] = np.nan
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                seq_data[f'tof_{tof_num}_band{band_idx}_mean'] = np.nanmean(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_std'] = np.nanstd(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_var'] = np.nanvar(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_min'] = np.nanmin(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_max'] = np.nanmax(band_data, axis=1)
                
    # ToF band statistics for tof_3 (statistically significant bands)
    bands_tof_3 = [
        (0, 47),    
        (50, 55)
    ]
    
    for tof_num in [3]:  # Only process tof_3
        for band_idx, (start, end) in enumerate(bands_tof_3, 1):
            # Get columns for this band (inclusive range)
            band_cols = [f"tof_{tof_num}_v{p}" for p in range(start, end + 1)]
            band_data = seq_data[band_cols].values
            band_data[band_data == -1] = np.nan
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                seq_data[f'tof_{tof_num}_band{band_idx}_mean'] = np.nanmean(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_std'] = np.nanstd(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_var'] = np.nanvar(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_min'] = np.nanmin(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_max'] = np.nanmax(band_data, axis=1)
                
    # ToF band statistics for tof_4 (statistically significant bands)
    bands_tof_4 = [
        (0, 3),    
        (7, 9),
        (15, 16),
        (21, 23),
        (28, 31),
        (35, 39),
        (43, 47),
        (50, 55),
        (58, 63)
    ]
    
    for tof_num in [4]:  # Only process tof_4
        for band_idx, (start, end) in enumerate(bands_tof_4, 1):
            # Get columns for this band (inclusive range)
            band_cols = [f"tof_{tof_num}_v{p}" for p in range(start, end + 1)]
            band_data = seq_data[band_cols].values
            band_data[band_data == -1] = np.nan
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                seq_data[f'tof_{tof_num}_band{band_idx}_mean'] = np.nanmean(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_std'] = np.nanstd(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_var'] = np.nanvar(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_min'] = np.nanmin(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_max'] = np.nanmax(band_data, axis=1)
                
    # ToF band statistics for tof_5 (statistically significant bands)
    bands_tof_5 = [
        (1, 7),    
        (9, 15),
        (18, 23),
        (48, 49),
        (56, 61)  
    ]
    
    for tof_num in [5]:  # Only process tof_5
        for band_idx, (start, end) in enumerate(bands_tof_5, 1):
            # Get columns for this band (inclusive range)
            band_cols = [f"tof_{tof_num}_v{p}" for p in range(start, end + 1)]
            band_data = seq_data[band_cols].values
            band_data[band_data == -1] = np.nan
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                seq_data[f'tof_{tof_num}_band{band_idx}_mean'] = np.nanmean(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_std'] = np.nanstd(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_var'] = np.nanvar(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_min'] = np.nanmin(band_data, axis=1)
                seq_data[f'tof_{tof_num}_band{band_idx}_max'] = np.nanmax(band_data, axis=1)            
    
    return seq_data

In [21]:
def load_data():
    # Load data
    df = pd.read_csv('input/train.csv', low_memory=False)
    train_dem_df = pd.read_csv('input/train_demographics.csv', low_memory=False)
    df = pd.merge(df, train_dem_df, on='subject', how='left')
    
    print("Encoding gestures...")
    # Encode labels
    le = LabelEncoder()
    df['gesture_int'] = le.fit_transform(df['gesture'])
    gesture_classes = le.classes_
    np.save(model_output_path / "gesture_classes.npy", gesture_classes)
    
    print("Done")
    
    print("Processing sequences with parallel computation...")
    
    # Group by sequence
    sequences = [group for _, group in df.groupby('sequence_id')]
    
    # Process in parallel
    n_cores = multiprocessing.cpu_count()
    print(f"Using {n_cores} cores for parallel processing")
    
    processed_sequences = Parallel(n_jobs=n_cores)(
        delayed(process_sequence)(seq) for seq in sequences
    )

    # Combine processed sequences
    df = pd.concat(processed_sequences, ignore_index=True)
    
    return df

processed_df = load_data()

Encoding gestures...
Done
Processing sequences with parallel computation...
Using 8 cores for parallel processing


In [7]:
# feature testing
def feature_engineering(df):   
    df = df.copy()

    # checks N/A before combining
    def combine_or_na(a, b):
        if 'N/A' in str(a) or 'N/A' in str(b):
            return 'N/A'
        return f"{a}_{b}"

    # combine orientation and gesture
    df['orientation_gesture'] = df.apply(lambda x: combine_or_na(x['orientation'], x['gesture']), axis=1).astype('category') 
        
    # behavioural boolean columns
    df['performs_gesture'] = df['behavior'].str.contains('Performs gesture', case=False, na=False)
    df['move_hand_to_target'] = df['behavior'].str.contains('Moves hand to target location', case=False, na=False)
    df['hand_at_target'] = df['behavior'].str.contains('Hand at target location', case=False, na=False)
    df['relaxes_moves_hand_to_target'] = df['behavior'].str.contains('Relaxes and moves hand to target location', case=False, na=False)
    
    return df

fe_train_df = feature_engineering(processed_df)

In [8]:
def fill_missing_values(df):
    # Fill categorical columns with 'N/A'
    cat_cols = df.select_dtypes(include=['object']).columns
    df[cat_cols] = df[cat_cols].fillna("N/A")
    
    # Fill all numerical columns with 0 (int8, int16, int32, int64, uint8, uint16, uint32, uint64, float16, float32, float64, complex64, complex128)
    num_cols = df.select_dtypes(include=[np.number]).columns  
    df[num_cols] = df[num_cols].replace([np.inf, -np.inf, '', None], np.nan).fillna(0)
    
    return df

fe_train_df  = fill_missing_values(fe_train_df )

In [9]:
# Get categorical columns
cat_features = fe_train_df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
cat_features

['row_id',
 'sequence_type',
 'sequence_id',
 'subject',
 'orientation',
 'behavior',
 'phase',
 'gesture',
 'orientation_gesture',
 'performs_gesture',
 'move_hand_to_target',
 'hand_at_target',
 'relaxes_moves_hand_to_target']

In [10]:
# Dictionary to store all encoder mappings
encoder_mappings = {}

print("Label Encoding Categorical Features: ",end="")
for c in cat_features:
    print(f"{c}, ",end="")
    fe_train_df[c] =  fe_train_df[c].astype('category')
    
    le = LabelEncoder()
    fe_train_df[c] = le.fit_transform(fe_train_df[c].astype(str))

    # Save the mapping of numerical values to string labels
    encoder_mappings[c] = {
        i: label for i, label in enumerate(le.classes_)
    }

with open('new_categorical_encoder_mappings.json', 'w') as f:
    json.dump(encoder_mappings, f, indent=2)

Label Encoding Categorical Features: row_id, sequence_type, sequence_id, subject, orientation, behavior, phase, gesture, orientation_gesture, performs_gesture, move_hand_to_target, hand_at_target, relaxes_moves_hand_to_target, 

In [11]:
# numerical distributions
def plot_numerical_distributions(df, output_path, exclude_zeros=False):
    numerical_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns
    
    # Create progress bar for numerical distributions
    for col in tqdm(numerical_cols, desc="Creating distribution plots"):
        plt.figure(figsize=(10, 8))
        
        # Filter data for plotting if exclude_zeros is True
        if exclude_zeros:
            plot_data = df[df[col] != 0][col]
            title_suffix = " (excluding zeros)"
        else:
            plot_data = df[col]
            title_suffix = ""
        
        # Skip if no data remains after filtering
        if len(plot_data) == 0:
            print(f"Warning: No data remaining for {col} after excluding zeros")
            plt.close()
            continue
        
        # Create subplot with histogram and kde
        sns.histplot(data=plot_data, kde=True)
        plt.title(f'Distribution of {col}{title_suffix}')
        plt.xlabel(col)
        plt.ylabel('Count')
        
        # Add statistical annotations based on filtered data
        stats_text = f'Mean: {plot_data.mean():.2f}\n'
        stats_text += f'Median: {plot_data.median():.2f}\n'
        stats_text += f'Std: {plot_data.std():.2f}\n'
        stats_text += f'Count: {len(plot_data)}'
        
        # Add additional info if zeros were excluded
        if exclude_zeros:
            zero_count = (df[col] == 0).sum()
            stats_text += f'\nZeros excluded: {zero_count}'
        
        plt.text(0.95, 0.95, stats_text,
                transform=plt.gca().transAxes,
                verticalalignment='top',
                horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        
        # Modify filename if zeros are excluded
        filename_suffix = "_no_zeros" if exclude_zeros else ""
        plt.savefig(f'{output_path}/distribution_{col}{filename_suffix}.png')
        plt.close()

# Exclude zeros from visualization
# plot_numerical_distributions(train_df, plot_output_path, exclude_zeros=False)

In [12]:
# categorical distributions
def plot_categorical_distributions(df, output_path):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    
    # Create progress bar for categorical distributions
    for col in tqdm(categorical_cols, desc="Creating distribution plots"):
        # Skip columns with 100 or more unique values
        if df[col].nunique() >= 100:
            print(f"Skipping {col}: too many unique values ({df[col].nunique()})")
            continue
            
        plt.figure(figsize=(18, 10))
        
        plot_data = df[col]
        
        # Skip if no data remains
        if len(plot_data) == 0:
            print(f"Warning: No data remaining for {col}")
            plt.close()
            continue
        
        # Create subplot with count plot
        sns.countplot(data=plot_data.to_frame(), y=col, order=plot_data.value_counts().index)
        plt.title(f'Distribution of {col}')
        plt.xlabel('Count')
        plt.ylabel(col)
        
        # Add statistical annotations
        stats_text = f'Unique: {plot_data.nunique()}\n'
        stats_text += f'Top freq: {plot_data.value_counts().iloc[0] if len(plot_data.value_counts()) > 0 else 0}\n'
        stats_text += f'Count: {len(plot_data)}'
        
        plt.text(0.95, 0.10, stats_text,
                transform=plt.gca().transAxes,
                verticalalignment='top',
                horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        
        plt.savefig(f'{output_path}/distribution_{col}.png')
        plt.close()

# Plot categorical distributions
# plot_categorical_distributions(train_df, plot_output_path)

In [13]:
# Ttest analysis
def perform_ttest_analysis(df, group_col, group1_value, group2_value, output_path):  
    # Filter data for the two groups
    group1_data = df[df[group_col] == group1_value].copy()
    group2_data = df[df[group_col] == group2_value].copy()
    
    print(f"Group 1 ({group1_value}): {len(group1_data)} records")
    print(f"Group 2 ({group2_value}): {len(group2_data)} records")
    
    # Get numerical columns
    numerical_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
    
    # Remove the group column if it's numerical
    if group_col in numerical_cols:
        numerical_cols.remove(group_col)
    
    print(f"Testing {len(numerical_cols)} numerical variables")
    
    # Initialize results list
    results = []
    
    # Perform t-tests for each numerical column
    for col in tqdm(numerical_cols, desc="Running t-tests and creating plots"):
        try:
            # Get data for both groups for this column (remove NaN values)
            group1_values = group1_data[col].dropna()
            group2_values = group2_data[col].dropna()
            
            # Skip if insufficient data
            if len(group1_values) < 2 or len(group2_values) < 2:
                print(f"Warning: Insufficient data for {col} (Group1: {len(group1_values)}, Group2: {len(group2_values)})")
                continue
            
            # Perform Levene's test for equal variances
            levene_stat, levene_p = stats.levene(group1_values, group2_values)
            equal_var = levene_p > 0.05
            
            # Perform independent t-test
            t_stat, p_value = stats.ttest_ind(group1_values, group2_values, equal_var=equal_var)
            
            # Calculate effect size (Cohen's d)
            if equal_var:
                pooled_std = np.sqrt(((len(group1_values) - 1) * group1_values.var() + 
                                     (len(group2_values) - 1) * group2_values.var()) / 
                                    (len(group1_values) + len(group2_values) - 2))
            else:
                pooled_std = np.sqrt((group1_values.var() + group2_values.var()) / 2)
            
            cohens_d = (group1_values.mean() - group2_values.mean()) / pooled_std if pooled_std != 0 else 0
            
            # Calculate 95% confidence interval for the difference in means
            diff_mean = group2_values.mean() - group1_values.mean()
            se_diff = np.sqrt(group1_values.var()/len(group1_values) + group2_values.var()/len(group2_values))
            t_critical = stats.t.ppf(0.975, len(group1_values) + len(group2_values) - 2)
            ci_lower = diff_mean - t_critical * se_diff
            ci_upper = diff_mean + t_critical * se_diff
            
            # Store results
            results.append({
                'variable': col,
                'group1_mean': group1_values.mean(),
                'group1_std': group1_values.std(),
                'group1_median': group1_values.median(),
                'group1_count': len(group1_values),
                'group2_mean': group2_values.mean(),
                'group2_std': group2_values.std(),
                'group2_median': group2_values.median(),
                'group2_count': len(group2_values),
                'mean_difference': diff_mean,
                'ci_lower': ci_lower,
                'ci_upper': ci_upper,
                't_statistic': t_stat,
                'p_value': p_value,
                'cohens_d': cohens_d,
                'equal_variances': equal_var,
                'levene_p_value': levene_p,
                'significant_005': p_value < 0.05,
                'significant_001': p_value < 0.01,
                'significant_0001': p_value < 0.001
            })
            
            # Create comparison plot
            create_comparison_plot(group_col, group1_values, group2_values, col, 
                                 group1_value, group2_value, 
                                 t_stat, p_value, cohens_d, output_path)
            
        except Exception as e:
            print(f"Error processing {col}: {str(e)}")
            continue
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    if len(results_df) > 0:
        # Sort by p-value
        results_df = results_df.sort_values('p_value').reset_index(drop=True)
        
        # Add interpretation columns
        results_df['effect_size_interpretation'] = results_df['cohens_d'].abs().apply(interpret_cohens_d)
        results_df['significance_level'] = results_df.apply(get_significance_level, axis=1)
    
    return results_df

def interpret_cohens_d(d):
    """Interpret Cohen's d effect size"""
    d_abs = abs(d)
    if d_abs < 0.2:
        return "negligible"
    elif d_abs < 0.5:
        return "small"
    elif d_abs < 0.8:
        return "medium"
    else:
        return "large"

def get_significance_level(row):
    """Get significance level string"""
    if row['p_value'] < 0.001:
        return "*** (p < 0.001)"
    elif row['p_value'] < 0.01:
        return "** (p < 0.01)"
    elif row['p_value'] < 0.05:
        return "* (p < 0.05)"
    else:
        return "Not significant"

def create_comparison_plot(group_col, group1_values, group2_values, col_name, 
                          group1_name, group2_name, t_stat, p_value, cohens_d, output_path):
    """Create comprehensive comparison plot for two groups"""
    
    # Set up the figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Statistical Comparison: {col_name}', fontsize=16, fontweight='bold')
    
    # Color scheme
    color1 = '#ff7f7f'  # Light red for group 1
    color2 = '#7fbfff'  # Light blue for group 2
    
    # 1. Histogram comparison (top left)
    ax1 = axes[0, 0]
    
    # Calculate bins for better visualization
    all_values = np.concatenate([group1_values, group2_values])
    bins = np.histogram_bin_edges(all_values, bins=30)
    
    ax1.hist(group1_values, bins=bins, alpha=0.7, label=f'{group1_name} (n={len(group1_values)})', 
             color=color1, density=True, edgecolor='black', linewidth=0.5)
    ax1.hist(group2_values, bins=bins, alpha=0.7, label=f'{group2_name} (n={len(group2_values)})', 
             color=color2, density=True, edgecolor='black', linewidth=0.5)
    
    # Add mean lines
    ax1.axvline(group1_values.mean(), color='darkred', linestyle='--', linewidth=2, 
                label=f'{group1_name} Mean: {group1_values.mean():.2f}')
    ax1.axvline(group2_values.mean(), color='darkblue', linestyle='--', linewidth=2,
                label=f'{group2_name} Mean: {group2_values.mean():.2f}')
    
    ax1.set_title('Distribution Comparison (Histogram)')
    ax1.set_xlabel(col_name)
    ax1.set_ylabel('Density')
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # 2. Box plot comparison (top right)
    ax2 = axes[0, 1]
    
    box_data = [group1_values, group2_values]
    box_labels = [f'{group1_name}\n(n={len(group1_values)})', f'{group2_name}\n(n={len(group2_values)})']
    
    bp = ax2.boxplot(box_data, labels=box_labels, patch_artist=True, 
                     boxprops=dict(alpha=0.7), showfliers=True)
    bp['boxes'][0].set_facecolor(color1)
    bp['boxes'][1].set_facecolor(color2)
    
    ax2.set_title('Box Plot Comparison')
    ax2.set_ylabel(col_name)
    ax2.grid(True, alpha=0.3)
    
    # 3. KDE plot (bottom left)
    ax3 = axes[1, 0]
    
    try:
        # Only plot KDE if we have enough data points
        if len(group1_values) > 3:
            sns.kdeplot(data=group1_values, label=f'{group1_name}', 
                       color='darkred', ax=ax3, linewidth=2)
        if len(group2_values) > 3:
            sns.kdeplot(data=group2_values, label=f'{group2_name}', 
                       color='darkblue', ax=ax3, linewidth=2)
    except:
        # Fallback to simple line plot if KDE fails
        ax3.hist(group1_values, alpha=0.5, density=True, color=color1, label=group1_name)
        ax3.hist(group2_values, alpha=0.5, density=True, color=color2, label=group2_name)
    
    ax3.set_title('Density Comparison (KDE)')
    ax3.set_xlabel(col_name)
    ax3.set_ylabel('Density')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Statistics summary (bottom right)
    ax4 = axes[1, 1]
    
    # Calculate additional statistics
    effect_size_interp = interpret_cohens_d(cohens_d)
    sig_level = get_significance_level({'p_value': p_value})
    
    stats_text = f'T-Test Results:\n'
    stats_text += f'━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n'
    stats_text += f'T-statistic: {t_stat:.4f}\n'
    stats_text += f'P-value: {p_value:.2e}\n'
    stats_text += f'Cohen\'s d: {cohens_d:.4f} ({effect_size_interp})\n'
    stats_text += f'Significance: {sig_level}\n\n'
    
    stats_text += f'{group1_name}:\n'
    stats_text += f'  Mean ± SD: {group1_values.mean():.2f} ± {group1_values.std():.2f}\n'
    stats_text += f'  Median: {group1_values.median():.2f}\n'
    stats_text += f'  Range: [{group1_values.min():.2f}, {group1_values.max():.2f}]\n'
    stats_text += f'  Count: {len(group1_values)}\n\n'
    
    stats_text += f'{group2_name}:\n'
    stats_text += f'  Mean ± SD: {group2_values.mean():.2f} ± {group2_values.std():.2f}\n'
    stats_text += f'  Median: {group2_values.median():.2f}\n'
    stats_text += f'  Range: [{group2_values.min():.2f}, {group2_values.max():.2f}]\n'
    stats_text += f'  Count: {len(group2_values)}\n\n'
    
    # Difference
    mean_diff = group2_values.mean() - group1_values.mean()
    stats_text += f'Mean Difference: {mean_diff:.2f}\n'
    stats_text += f'({group2_name} - {group1_name})\n\n'
    
    # Effect size interpretation
    stats_text += f'Effect Size Interpretation:\n'
    if abs(cohens_d) < 0.2:
        stats_text += f'Negligible practical difference'
    elif abs(cohens_d) < 0.5:
        stats_text += f'Small practical difference'
    elif abs(cohens_d) < 0.8:
        stats_text += f'Medium practical difference'
    else:
        stats_text += f'Large practical difference'
    
    ax4.text(0.05, 0.95, stats_text,
             transform=ax4.transAxes,
             verticalalignment='top',
             horizontalalignment='left',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8),
             fontsize=10, fontfamily='monospace')
    ax4.set_xlim(0, 1)
    ax4.set_ylim(0, 1)
    ax4.axis('off')
    
    plt.tight_layout()
    
    # Save with significance indicator in filename
    if p_value < 0.001:
        sig_suffix = "_highly_significant"
    elif p_value < 0.05:
        sig_suffix = "_significant"
    else:
        sig_suffix = "_not_significant"
    
    plt.savefig(f'{output_path}/{group_col}_{col_name}_ttest_{group1_name}_vs_{group2_name}_{sig_suffix}_{effect_size_interp}.png', 
                dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()


# Perform the analysis
#ttest_results = perform_ttest_analysis(
#    df=fe_train_df, 
#    group_col='performs_gesture',
#    group1_value=True,
#    group2_value=False,
#    output_path=plot_output_path
#)

Group 1 (True): 255817 records
Group 2 (False): 319128 records
Testing 406 numerical variables


Running t-tests and creating plots: 100%|██████████| 406/406 [1:23:07<00:00, 12.29s/it] 
